In [7]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import time

In [8]:
#webcam 
cap=cv2.VideoCapture(0)
cap.set(3,1920) #width
cap.set(4,1080) #height
detector=HandDetector(detectionCon=0.5,maxHands=1)

In [13]:
#creating button class
class Button:
    def __init__(self,pos,width,height,text):
        self.pos=pos
        self.width=width
        self.height=height
        self.text=text
    
    def draw(self,img):
        cv2.rectangle(img, self.pos, (self.pos[0]+self.width,self.pos[1]+self.height), (240, 240, 240), cv2.FILLED) # Draw a rectangle around the image
        cv2.rectangle(img, self.pos, (self.pos[0]+self.width,self.pos[1]+self.height), (100,100,100), 2) # Draw a rectangle around the image
        (text_width, text_height), _ = cv2.getTextSize(self.text, cv2.FONT_HERSHEY_PLAIN, 2, 2)
        text_x = self.pos[0] + (self.width - text_width) // 2
        text_y = self.pos[1] + (self.height + text_height) // 2
        if self.text in ['+', '-', '*', '/', '=']:
            # Operators - blue
            cv2.putText(img, self.text, (text_x, text_y), cv2.FONT_HERSHEY_PLAIN, 2, (0, 100, 200), 2)
        elif self.text in ['C', 'CE', 'DEL', '%']:
            # Special functions - red
            cv2.putText(img, self.text, (text_x, text_y), cv2.FONT_HERSHEY_PLAIN, 2, (200, 50, 50), 2)
        else:
            # Numbers and decimal - dark gray
            cv2.putText(img, self.text, (text_x, text_y), cv2.FONT_HERSHEY_PLAIN, 2, (50, 50, 50), 2)
        
    def click(self,x,y):
        if self.pos[0]<x<self.pos[0]+self.width and self.pos[1]<y<self.pos[1]+self.height:
            # Pressed state - darker background
            cv2.rectangle(img, self.pos, (self.pos[0]+self.width,self.pos[1]+self.height), (200, 200, 200), cv2.FILLED)
            cv2.rectangle(img, self.pos, (self.pos[0]+self.width,self.pos[1]+self.height), (80, 80, 80), 2)
            
            # Center the text
            (text_width, text_height), _ = cv2.getTextSize(self.text, cv2.FONT_HERSHEY_PLAIN, 4, 4)
            text_x = self.pos[0] + (self.width - text_width) // 2
            text_y = self.pos[1] + (self.height + text_height) // 2
            
            # White text for pressed state
            cv2.putText(img, self.text, (text_x, text_y), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
            return True
        else:
            return False

In [17]:
#create a button
button_list_values = [["%","CE","C","DEL"],
                      ["7","8","9","*"],
                      ["4","5","6","-"],
                      ["1","2","3","+"],
                      ["/","0",".","="],]

button_list = []
for i in range(4):
    for j in range(5):
        xpos = i*75 + 300
        ypos = j*75 + 125
        button_list.append(Button((xpos,ypos),75,75,button_list_values[j][i]))
myEquation = ""
delaycounter = 0


In [18]:
#loop to capture video
while True:
    #get image from camera
    success,img=cap.read()
    img = cv2.flip(img, 1) # Flip the image horizontally
    #create hand detector object
    hands,img= detector.findHands(img,flipType=False) #flipType=False to avoid flipping the image again 
    
    #drew the button
    cv2.rectangle(img, (300,50), (300+300,50+75), (250, 250, 250), cv2.FILLED)
    cv2.rectangle(img, (300,50), (300+300,50+75), (80,80,80), 3)
    for button in button_list:
        button.draw(img)
    
    #check if hands are detected
    if hands:
        lmList = hands[0]["lmList"] #get the landmark list of the first hand
        length, _, img = detector.findDistance(lmList[4][:2], lmList[8][:2], img) #find the distance between the index finger and thumb 
        x,y = lmList[8][:2]
        if length<50:
            for i,button in enumerate(button_list):
                if button.click(x,y) and delaycounter==0:
                    myValue = button_list_values[int(i%5)][int(i//5)]
                    if myValue == "=":
                        myEquation = str(eval(myEquation))
                    elif myValue == "C":
                        myEquation = ""
                    elif myValue == "DEL":
                        myEquation = myEquation[:-1]
                    elif myValue == "CE":
                        myEquation = ""
                    
                    else:
                        myEquation += myValue
                    delaycounter=1
                    time.sleep(0.2)
    #avoid duplicates
    if delaycounter!=0:
        delaycounter+=1
        if delaycounter>10:
            delaycounter=0
    
    #output screen
    display_width = 300
    display_x = 300
    (text_width, text_height), _ = cv2.getTextSize(myEquation, cv2.FONT_HERSHEY_PLAIN, 2, 2)
    text_x = display_x + (display_width - text_width ) //2
    text_y = 50 + (75 + text_height) // 2
    cv2.putText(img, myEquation, (text_x, text_y), cv2.FONT_HERSHEY_PLAIN, 2, (30, 30, 30), 2) 
    
    
    #display the image
    cv2.imshow("Image", img)
    cv2.waitKey(1)
    

KeyboardInterrupt: 